# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

In [0]:
import tensorflow as tf


#### Import the mnist dataset from keras datasets

In [0]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [7]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [8]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0],28,28,1)
x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0],28,28,1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')

#Normalizing the input
x_train_lt5 /= 255
x_test_lt5 /= 255


### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [16]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
#import keras
y_train_lt5 = tf.keras.utils.to_categorical(y_train_lt5,5)

In [0]:
y_test_lt5 = tf.keras.utils.to_categorical(y_test_lt5,5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [24]:
import keras

Using TensorFlow backend.


In [0]:
from keras  import Sequential

In [0]:
model = Sequential()

In [0]:
from keras.layers import Conv2D, MaxPooling2D
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))

In [0]:
model.add(MaxPooling2D(pool_size=(2,2))) #pooling layer

In [0]:
#drop out layer
from keras.layers import Dense, Dropout, Flatten
model.add(Dropout(.25))


### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(5,activation='softmax'))

### Print the training and test accuracy for 5 epochs

In [0]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [65]:
model.fit(x_train_lt5,y_train_lt5,batch_size=50,epochs=5,validation_data=(x_test_lt5, y_test_lt5))

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 7s 244us/step - loss: 0.0784 - acc: 0.9763 - val_loss: 0.0113 - val_acc: 0.9961
Epoch 2/5
30596/30596 [==============================] - 7s 218us/step - loss: 0.0218 - acc: 0.9931 - val_loss: 0.0270 - val_acc: 0.9920
Epoch 3/5
30596/30596 [==============================] - 7s 216us/step - loss: 0.0148 - acc: 0.9952 - val_loss: 0.0119 - val_acc: 0.9963
Epoch 4/5
30596/30596 [==============================] - 7s 215us/step - loss: 0.0095 - acc: 0.9967 - val_loss: 0.0049 - val_acc: 0.9986
Epoch 5/5
30596/30596 [==============================] - 7s 214us/step - loss: 0.0066 - acc: 0.9980 - val_loss: 0.0063 - val_acc: 0.9981


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [66]:
model.layers

In [67]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv2d_11
conv2d_11is not trainable

conv2d_12
conv2d_12is not trainable

max_pooling2d_4
max_pooling2d_4is not trainable

dropout_7
dropout_7is not trainable

flatten_4
flatten_4is not trainable

dense_7
dense_7 is trainable

dropout_8
dropout_8is not trainable

dense_8
dense_8 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0], 28, 28, 1)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0], 28, 28, 1)
x_train_gt5 = x_train_gt5.astype('float32')/255
x_test_gt5 = x_test_gt5.astype('float32')/255
y_train_gt5 = keras.utils.to_categorical(y_train_gt5, 5)
y_test_gt5 = keras.utils.to_categorical(y_test_gt5, 5)




### Check

Verify the shapes with the given below.

In [70]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [71]:
#Training on the dataset
model.fit(x_train_gt5, y_train_gt5,
          epochs=5,
          validation_data=(x_test_gt5, y_test_gt5))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
  416/29404 [..............................] - ETA: 13s - loss: 1.8322 - acc: 0.6130

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 9s 305us/step - loss: 0.1058 - acc: 0.9690 - val_loss: 0.0294 - val_acc: 0.9903
Epoch 2/5
29404/29404 [==============================] - 9s 302us/step - loss: 0.0323 - acc: 0.9895 - val_loss: 0.0271 - val_acc: 0.9899
Epoch 3/5
29404/29404 [==============================] - 9s 304us/step - loss: 0.0214 - acc: 0.9930 - val_loss: 0.0244 - val_acc: 0.9936
Epoch 4/5
29404/29404 [==============================] - 9s 305us/step - loss: 0.0168 - acc: 0.9942 - val_loss: 0.0252 - val_acc: 0.9918
Epoch 5/5
29404/29404 [==============================] - 9s 306us/step - loss: 0.0130 - acc: 0.9958 - val_loss: 0.0217 - val_acc: 0.9928


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:

from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [74]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [76]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [77]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [78]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [0]:
vect = TfidfVectorizer()

In [0]:
x_train_dm = vect.fit_transform(twenty_train.data)
x_test_dm = vect.transform(twenty_test.data)


In [92]:
import pandas as pd
pd.DataFrame(x_train_dm.toarray(), columns=vect.get_feature_names())

,00,000,0000,0000001200,000005102000,0001,000100255pixel,00014,000406,0007,000usd,0010,001004,0010580b,001125,001200201pixel,0014,001642,00196,002,0028,003258u19250,0033,0038,0039,004021809,004158,004627,0049,00500,005148,00630,008561,0094,00am,00index,00pm,01,0100,010116,...,zoerasterism,zola,zolf,zolft,zoloft,zombie,zonal,zone,zonker,zoo,zooid,zool,zoom,zooming,zooms,zopfi,zorasterism,zorg,zorn,zrz,zsoft,zt,zubkoff,zues,zug,zumder,zur,zurich,zurlo,zus,zvi,zvonko,zwart,zyeh,zyklon,zyxel,zz,zzz,ªl,íålittin
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.29521,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.055283,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057185,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.095199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2253,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2254,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2255,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

y_train_dm =twenty_train.target
y_test_dm=twenty_test.target

logreg = LogisticRegression(C=1e9)
logreg.fit(x_train_dm, y_train_dm)

y_train_pred_log=logreg.predict(x_train_dm)
print('Accuracy for Train by Logistic: ', metrics.accuracy_score(y_train_dm, y_train_pred_log))

y_pred_class = logreg.predict(x_test_dm)
print('Accuracy for Test by Logistic: ',metrics.accuracy_score(y_test_dm, y_pred_class))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy for Train by Logistic:  1.0
Accuracy for Test by Logistic:  0.9260985352862849
